In [1]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

os.makedirs("images", exist_ok=True)

In [2]:
#parser = argparse.ArgumentParser()
#parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")
#parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
#parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
#parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
#parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
#parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
#parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
#parser.add_argument("--n_classes", type=int, default=10, help="number of classes for dataset")
#parser.add_argument("--img_size", type=int, default=32, help="size of each image dimension")
#parser.add_argument("--channels", type=int, default=1, help="number of image channels")
#parser.add_argument("--sample_interval", type=int, default=400, help="interval between image sampling")
#opt = parser.parse_args()
#print(opt)

In [3]:
class parameter_opt():
    None
opt = parameter_opt()

In [4]:
opt.n_epochs=200
opt.batch_size = 64
opt.lr = 0.0002
opt.b1 = 0.5
opt.b2 = 0.999
opt.n_cpu = 8
opt.latent_dim = 100
opt.n_classes = 27
opt.img_size = 64
opt.channels = 3
opt.sample_interval = 5000

In [5]:
img_shape = (opt.channels, opt.img_size, opt.img_size)

cuda = True if torch.cuda.is_available() else False

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(opt.n_classes, opt.n_classes)

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(opt.latent_dim + opt.n_classes, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(opt.n_classes, opt.n_classes)

        self.model = nn.Sequential(
            nn.Linear(opt.n_classes + int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity

In [7]:
# Loss functions
adversarial_loss = torch.nn.MSELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [8]:
## Configure data loader
#os.makedirs("../../data/mnist", exist_ok=True)
#dataloader = torch.utils.data.DataLoader(
#    datasets.MNIST(
#        "../../data/mnist",
#        train=True,
#        download=True,
#        transform=transforms.Compose(
#            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
#        ),
#    ),
#    batch_size=opt.batch_size,
#    shuffle=True,
#)

from dataloader_wikiart import *

dataloader = get_dataset()

In [9]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

In [10]:
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, opt.latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    gen_imgs = generator(z, labels)
    save_image(gen_imgs.data, "images/%d.png" % batches_done, nrow=n_row, normalize=True)

In [ ]:
# ----------
#  Training
# ----------

for epoch in range(opt.n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(labels.type(LongTensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, opt.latent_dim))))
        gen_labels = Variable(LongTensor(np.random.randint(0, opt.n_classes, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity = discriminator(gen_imgs, gen_labels)
        g_loss = adversarial_loss(validity, valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        validity_real = discriminator(real_imgs, labels)
        d_real_loss = adversarial_loss(validity_real, valid)

        # Loss for fake images
        validity_fake = discriminator(gen_imgs.detach(), gen_labels)
        d_fake_loss = adversarial_loss(validity_fake, fake)

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            sample_image(n_row=27, batches_done=batches_done)

[Epoch 0/200] [Batch 0/637] [D loss: 0.501207] [G loss: 1.025522]
[Epoch 0/200] [Batch 1/637] [D loss: 0.237407] [G loss: 0.984633]
[Epoch 0/200] [Batch 2/637] [D loss: 0.155929] [G loss: 0.934533]
[Epoch 0/200] [Batch 3/637] [D loss: 0.156103] [G loss: 0.901976]
[Epoch 0/200] [Batch 4/637] [D loss: 0.105348] [G loss: 0.872902]
[Epoch 0/200] [Batch 5/637] [D loss: 0.128893] [G loss: 0.843888]
[Epoch 0/200] [Batch 6/637] [D loss: 0.076811] [G loss: 0.814623]
[Epoch 0/200] [Batch 7/637] [D loss: 0.091339] [G loss: 0.771831]
[Epoch 0/200] [Batch 8/637] [D loss: 0.068928] [G loss: 0.736516]
[Epoch 0/200] [Batch 9/637] [D loss: 0.081164] [G loss: 0.703316]
[Epoch 0/200] [Batch 10/637] [D loss: 0.101053] [G loss: 0.696367]
[Epoch 0/200] [Batch 11/637] [D loss: 0.080394] [G loss: 0.673285]
[Epoch 0/200] [Batch 12/637] [D loss: 0.096459] [G loss: 0.650023]
[Epoch 0/200] [Batch 13/637] [D loss: 0.073343] [G loss: 0.650755]
[Epoch 0/200] [Batch 14/637] [D loss: 0.085626] [G loss: 0.645087]
[Epoc

[Epoch 0/200] [Batch 126/637] [D loss: 0.088509] [G loss: 0.937730]
[Epoch 0/200] [Batch 127/637] [D loss: 0.089228] [G loss: 0.641958]
[Epoch 0/200] [Batch 128/637] [D loss: 0.110962] [G loss: 1.126111]
[Epoch 0/200] [Batch 129/637] [D loss: 0.083882] [G loss: 0.807143]
[Epoch 0/200] [Batch 130/637] [D loss: 0.088453] [G loss: 0.780978]
[Epoch 0/200] [Batch 131/637] [D loss: 0.066855] [G loss: 0.910092]
[Epoch 0/200] [Batch 132/637] [D loss: 0.078661] [G loss: 0.782249]
[Epoch 0/200] [Batch 133/637] [D loss: 0.064286] [G loss: 0.892181]
[Epoch 0/200] [Batch 134/637] [D loss: 0.064654] [G loss: 0.894287]
[Epoch 0/200] [Batch 135/637] [D loss: 0.073887] [G loss: 0.967721]
[Epoch 0/200] [Batch 136/637] [D loss: 0.082690] [G loss: 0.634182]
[Epoch 0/200] [Batch 137/637] [D loss: 0.084126] [G loss: 1.170176]
[Epoch 0/200] [Batch 138/637] [D loss: 0.072053] [G loss: 0.805461]
[Epoch 0/200] [Batch 139/637] [D loss: 0.075512] [G loss: 0.738780]
[Epoch 0/200] [Batch 140/637] [D loss: 0.075322]

[Epoch 0/200] [Batch 250/637] [D loss: 0.125007] [G loss: 0.669338]
[Epoch 0/200] [Batch 251/637] [D loss: 0.133701] [G loss: 0.542513]
[Epoch 0/200] [Batch 252/637] [D loss: 0.138317] [G loss: 0.885819]
[Epoch 0/200] [Batch 253/637] [D loss: 0.178930] [G loss: 0.596846]
[Epoch 0/200] [Batch 254/637] [D loss: 0.233834] [G loss: 0.785572]
[Epoch 0/200] [Batch 255/637] [D loss: 0.163957] [G loss: 0.552765]
[Epoch 0/200] [Batch 256/637] [D loss: 0.160765] [G loss: 0.705834]
[Epoch 0/200] [Batch 257/637] [D loss: 0.131709] [G loss: 0.582026]
[Epoch 0/200] [Batch 258/637] [D loss: 0.128528] [G loss: 0.567117]
[Epoch 0/200] [Batch 259/637] [D loss: 0.122660] [G loss: 0.667782]
[Epoch 0/200] [Batch 260/637] [D loss: 0.106882] [G loss: 0.606192]
[Epoch 0/200] [Batch 261/637] [D loss: 0.118104] [G loss: 0.647424]
[Epoch 0/200] [Batch 262/637] [D loss: 0.096786] [G loss: 0.640684]
[Epoch 0/200] [Batch 263/637] [D loss: 0.114506] [G loss: 0.857710]
[Epoch 0/200] [Batch 264/637] [D loss: 0.125544]

[Epoch 0/200] [Batch 374/637] [D loss: 0.127267] [G loss: 0.544775]
[Epoch 0/200] [Batch 375/637] [D loss: 0.113423] [G loss: 0.727485]
[Epoch 0/200] [Batch 376/637] [D loss: 0.115284] [G loss: 0.763331]
[Epoch 0/200] [Batch 377/637] [D loss: 0.101809] [G loss: 0.739656]
[Epoch 0/200] [Batch 378/637] [D loss: 0.104529] [G loss: 0.793503]
[Epoch 0/200] [Batch 379/637] [D loss: 0.126517] [G loss: 0.639211]
[Epoch 0/200] [Batch 380/637] [D loss: 0.116666] [G loss: 0.747875]
[Epoch 0/200] [Batch 381/637] [D loss: 0.108371] [G loss: 0.656006]
[Epoch 0/200] [Batch 382/637] [D loss: 0.115366] [G loss: 0.801808]
[Epoch 0/200] [Batch 383/637] [D loss: 0.125746] [G loss: 0.651897]
[Epoch 0/200] [Batch 384/637] [D loss: 0.124973] [G loss: 0.793035]
[Epoch 0/200] [Batch 385/637] [D loss: 0.114702] [G loss: 0.677938]
[Epoch 0/200] [Batch 386/637] [D loss: 0.124675] [G loss: 0.652258]
[Epoch 0/200] [Batch 387/637] [D loss: 0.094451] [G loss: 0.759600]
[Epoch 0/200] [Batch 388/637] [D loss: 0.162530]

[Epoch 0/200] [Batch 495/637] [D loss: 0.150446] [G loss: 0.654681]
[Epoch 0/200] [Batch 496/637] [D loss: 0.123491] [G loss: 0.680376]
[Epoch 0/200] [Batch 497/637] [D loss: 0.106150] [G loss: 0.754012]
[Epoch 0/200] [Batch 498/637] [D loss: 0.139943] [G loss: 0.655163]
[Epoch 0/200] [Batch 499/637] [D loss: 0.154724] [G loss: 0.520580]
[Epoch 0/200] [Batch 500/637] [D loss: 0.231054] [G loss: 1.286825]
[Epoch 0/200] [Batch 501/637] [D loss: 0.181112] [G loss: 0.540668]
[Epoch 0/200] [Batch 502/637] [D loss: 0.159193] [G loss: 0.546316]
[Epoch 0/200] [Batch 503/637] [D loss: 0.151456] [G loss: 0.596590]
[Epoch 0/200] [Batch 504/637] [D loss: 0.142940] [G loss: 0.567706]
[Epoch 0/200] [Batch 505/637] [D loss: 0.116235] [G loss: 0.578875]
[Epoch 0/200] [Batch 506/637] [D loss: 0.137303] [G loss: 0.725224]
[Epoch 0/200] [Batch 507/637] [D loss: 0.124281] [G loss: 0.722520]
[Epoch 0/200] [Batch 508/637] [D loss: 0.120925] [G loss: 0.654730]
[Epoch 0/200] [Batch 509/637] [D loss: 0.125192]

...

[Epoch 44/200] [Batch 95/637] [D loss: 0.170435] [G loss: 0.452744]
[Epoch 44/200] [Batch 96/637] [D loss: 0.153145] [G loss: 0.469938]
[Epoch 44/200] [Batch 97/637] [D loss: 0.162318] [G loss: 0.526001]
[Epoch 44/200] [Batch 98/637] [D loss: 0.185820] [G loss: 0.495505]
[Epoch 44/200] [Batch 99/637] [D loss: 0.153085] [G loss: 0.615410]
[Epoch 44/200] [Batch 100/637] [D loss: 0.149497] [G loss: 0.447104]
[Epoch 44/200] [Batch 101/637] [D loss: 0.175285] [G loss: 0.474213]
[Epoch 44/200] [Batch 102/637] [D loss: 0.162009] [G loss: 0.480930]
[Epoch 44/200] [Batch 103/637] [D loss: 0.178530] [G loss: 0.475752]
[Epoch 44/200] [Batch 104/637] [D loss: 0.177716] [G loss: 0.457809]
[Epoch 44/200] [Batch 105/637] [D loss: 0.197135] [G loss: 0.493903]
[Epoch 44/200] [Batch 106/637] [D loss: 0.177333] [G loss: 0.479394]
[Epoch 44/200] [Batch 107/637] [D loss: 0.170770] [G loss: 0.485185]
[Epoch 44/200] [Batch 108/637] [D loss: 0.197810] [G loss: 0.480346]
[Epoch 44/200] [Batch 109/637] [D loss:

[Epoch 44/200] [Batch 215/637] [D loss: 0.156796] [G loss: 0.506416]
[Epoch 44/200] [Batch 216/637] [D loss: 0.155428] [G loss: 0.464034]
[Epoch 44/200] [Batch 217/637] [D loss: 0.170021] [G loss: 0.473809]
[Epoch 44/200] [Batch 218/637] [D loss: 0.161918] [G loss: 0.502596]
[Epoch 44/200] [Batch 219/637] [D loss: 0.163804] [G loss: 0.471786]
[Epoch 44/200] [Batch 220/637] [D loss: 0.153787] [G loss: 0.511689]
[Epoch 44/200] [Batch 221/637] [D loss: 0.159307] [G loss: 0.526312]
[Epoch 44/200] [Batch 222/637] [D loss: 0.141328] [G loss: 0.538516]
[Epoch 44/200] [Batch 223/637] [D loss: 0.178490] [G loss: 0.516982]
[Epoch 44/200] [Batch 224/637] [D loss: 0.150785] [G loss: 0.576859]
[Epoch 44/200] [Batch 225/637] [D loss: 0.156556] [G loss: 0.549654]
[Epoch 44/200] [Batch 226/637] [D loss: 0.151797] [G loss: 0.479687]
[Epoch 44/200] [Batch 227/637] [D loss: 0.158998] [G loss: 0.482835]
[Epoch 44/200] [Batch 228/637] [D loss: 0.171759] [G loss: 0.490470]
[Epoch 44/200] [Batch 229/637] [D 

[Epoch 44/200] [Batch 339/637] [D loss: 0.155663] [G loss: 0.628645]
[Epoch 44/200] [Batch 340/637] [D loss: 0.179674] [G loss: 0.511452]
[Epoch 44/200] [Batch 341/637] [D loss: 0.166419] [G loss: 0.473367]
[Epoch 44/200] [Batch 342/637] [D loss: 0.171480] [G loss: 0.533668]
[Epoch 44/200] [Batch 343/637] [D loss: 0.157745] [G loss: 0.508130]
[Epoch 44/200] [Batch 344/637] [D loss: 0.164378] [G loss: 0.512613]
[Epoch 44/200] [Batch 345/637] [D loss: 0.195588] [G loss: 0.481383]
[Epoch 44/200] [Batch 346/637] [D loss: 0.166204] [G loss: 0.465844]
[Epoch 44/200] [Batch 347/637] [D loss: 0.172807] [G loss: 0.495571]
[Epoch 44/200] [Batch 348/637] [D loss: 0.150835] [G loss: 0.512426]
[Epoch 44/200] [Batch 349/637] [D loss: 0.162675] [G loss: 0.440577]
[Epoch 44/200] [Batch 350/637] [D loss: 0.161398] [G loss: 0.469208]
[Epoch 44/200] [Batch 351/637] [D loss: 0.179195] [G loss: 0.457718]
[Epoch 44/200] [Batch 352/637] [D loss: 0.167268] [G loss: 0.480073]
[Epoch 44/200] [Batch 353/637] [D 

[Epoch 44/200] [Batch 462/637] [D loss: 0.198579] [G loss: 0.435625]
[Epoch 44/200] [Batch 463/637] [D loss: 0.211995] [G loss: 0.476890]
[Epoch 44/200] [Batch 464/637] [D loss: 0.176666] [G loss: 0.594558]
[Epoch 44/200] [Batch 465/637] [D loss: 0.171861] [G loss: 0.565309]
[Epoch 44/200] [Batch 466/637] [D loss: 0.173898] [G loss: 0.473303]
[Epoch 44/200] [Batch 467/637] [D loss: 0.175983] [G loss: 0.448607]
[Epoch 44/200] [Batch 468/637] [D loss: 0.188240] [G loss: 0.390882]
[Epoch 44/200] [Batch 469/637] [D loss: 0.171524] [G loss: 0.488953]
[Epoch 44/200] [Batch 470/637] [D loss: 0.170991] [G loss: 0.533953]
[Epoch 44/200] [Batch 471/637] [D loss: 0.145856] [G loss: 0.500913]
[Epoch 44/200] [Batch 472/637] [D loss: 0.192194] [G loss: 0.474013]
[Epoch 44/200] [Batch 473/637] [D loss: 0.171662] [G loss: 0.548641]
[Epoch 44/200] [Batch 474/637] [D loss: 0.159703] [G loss: 0.511668]
[Epoch 44/200] [Batch 475/637] [D loss: 0.179495] [G loss: 0.396921]
[Epoch 44/200] [Batch 476/637] [D 

[Epoch 44/200] [Batch 586/637] [D loss: 0.181717] [G loss: 0.408912]
[Epoch 44/200] [Batch 587/637] [D loss: 0.160599] [G loss: 0.503081]
[Epoch 44/200] [Batch 588/637] [D loss: 0.165102] [G loss: 0.565180]
[Epoch 44/200] [Batch 589/637] [D loss: 0.183333] [G loss: 0.468130]
[Epoch 44/200] [Batch 590/637] [D loss: 0.145505] [G loss: 0.532032]
[Epoch 44/200] [Batch 591/637] [D loss: 0.162756] [G loss: 0.542068]
[Epoch 44/200] [Batch 592/637] [D loss: 0.165625] [G loss: 0.512518]
[Epoch 44/200] [Batch 593/637] [D loss: 0.159031] [G loss: 0.521477]
[Epoch 44/200] [Batch 594/637] [D loss: 0.178229] [G loss: 0.500051]
[Epoch 44/200] [Batch 595/637] [D loss: 0.156409] [G loss: 0.436877]
[Epoch 44/200] [Batch 596/637] [D loss: 0.180414] [G loss: 0.463848]
[Epoch 44/200] [Batch 597/637] [D loss: 0.150654] [G loss: 0.498891]
[Epoch 44/200] [Batch 598/637] [D loss: 0.179929] [G loss: 0.487984]
[Epoch 44/200] [Batch 599/637] [D loss: 0.179683] [G loss: 0.487475]
[Epoch 44/200] [Batch 600/637] [D 

In [16]:
gen_imgs.shape

torch.Size([729, 3, 64, 64])